In [ ]:
print("ok")

In [ ]:
%pip install openai twilio flask

In [ ]:
%pip install python-dotenv


In [ ]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client

# Set environment variables for your credentials
# Read more at http://twil.io/secure

account_sid = "AC25ff99962f5f1c10f4610882da6d8652"
auth_token = "1a65fada9d981730798f8f8c27d35f67"
client = Client(account_sid, auth_token)

call = client.calls.create(
  url="http://demo.twilio.com/docs/voice.xml",
  to="+923464056076",
  from_="+16813666079"
)

print(call.sid)

In [ ]:
import os
from dotenv import load_dotenv
from flask import Flask, request
from twilio.twiml.voice_response import VoiceResponse

# Load environment variables
load_dotenv()

# Get Twilio credentials from .env file
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_PHONE_NUMBER = os.getenv("TWILIO_PHONE_NUMBER")

app = Flask(__name__)

@app.route("/voice", methods=['POST'])
def voice():
    """Handle incoming voice calls"""
    response = VoiceResponse()
    response.say("Hello! Welcome to our AI Sales Agent. How can I assist you today?", voice='alice', language='en-US')
    
    # Record customer input and send it for processing
    response.record(timeout=5, transcribe=True, action="/process")
    
    return str(response)

@app.route("/process", methods=['POST'])
def process_voice():
    """Process the transcribed speech"""
    customer_speech = request.form.get("TranscriptionText", "Sorry, I couldn't understand.")
    
    response = VoiceResponse()
    response.say(f"You said: {customer_speech}. I will process your request shortly.", voice='alice')

    return str(response)

# if __name__ == "__main__":
#     app.run(port=5000, debug=True)


In [ ]:
%pip install pyngrok


https://3fb6-39-46-137-200.ngrok-free.app

In [ ]:
from twilio.rest import Client
import os
from dotenv import load_dotenv
from flask import Flask, request, jsonify

# Load environment variables
load_dotenv()

# Twilio credentials
TWILIO_ACCOUNT_SID = os.getenv("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
TWILIO_PHONE_NUMBER = os.getenv("TWILIO_PHONE_NUMBER")

client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

app = Flask(__name__)

@app.route("/make-call", methods=['POST'])
def make_call():
    """Initiate an outbound call"""
    data = request.json
    customer_number = data.get("customer_number")

    if not customer_number:
        return jsonify({"error": "Missing customer number"}), 400

    call = client.calls.create(
        to=customer_number,
        from_=TWILIO_PHONE_NUMBER,
        url="https://abc123.ngrok.io/outbound-voice"
    )

    return jsonify({"message": "Call initiated", "call_sid": call.sid})


@app.route("/outbound-voice", methods=['POST'])
def outbound_voice():
    """Handle outbound call responses"""
    from twilio.twiml.voice_response import VoiceResponse
    response = VoiceResponse()
    response.say("Hello! This is your AI Sales Agent calling. How can I assist you today?", voice='alice', language='en-US')
    
    return str(response)

if __name__ == "__main__":
    app.run(port=5000, debug=True)


In [ ]:
%pip install -U openai-whisper

In [ ]:
%pip install elevenlabs


In [ ]:
import whisper
from elevenlabs import generate, play

# Load Whisper model
def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"]

# Text-to-Speech function
def text_to_speech(text):
    audio = generate(text=text, voice="Adam", model="eleven_multilingual_v2")
    play(audio)

# Test transcription (Replace 'sample.wav' with an actual file path)
#audio_text = transcribe_audio("sample.wav")
#print("Transcribed Text:", audio_text)

# Test TTS
test_text = "Hello! How can I assist you today?"
text_to_speech(test_text)


In [1]:
print("feb 24")

feb 24


In [4]:
from flask import Flask, request
from twilio.twiml.voice_response import VoiceResponse

app = Flask(__name__)

@app.route("/voice", methods=["POST"])
def voice():
    """Handles incoming calls and responds with the company name"""
    response = VoiceResponse()
    response.say("Hello! Welcome to XYZ Company. How can I assist you today?", voice="alice")
    return str(response)

if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Feb/2025 14:48:59] "POST /voice HTTP/1.1" 200 -


In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# Initialize embeddings model
embeddings = OpenAIEmbeddings()

# Sample Business Knowledge
documents = [
    Document(page_content="Our product is a cloud-based CRM that helps manage customer interactions."),
    Document(page_content="We offer a 14-day free trial with no credit card required."),
    Document(page_content="Our pricing starts at $49 per month for basic features."),
]

# Convert documents into embeddings and store in FAISS
vector_store = FAISS.from_documents(documents, embeddings)
vector_store.save_local("faiss_index")


C:\Users\Waleed\AppData\Local\Temp\ipykernel_12992\1009501427.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [2]:


vector_store = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

def retrieve_info(query):
    results = vector_store.similarity_search(query, k=3)  # Retrieve top 3 relevant docs
    return "\n".join([doc.page_content for doc in results])


In [7]:
from flask import Flask, request, Response
from twilio.twiml.voice_response import VoiceResponse

app = Flask(__name__)

@app.route("/voice", methods=["POST"])
def voice():
    response = VoiceResponse()

    # Enable speech recognition
    response.say("Hello, welcome to [Your Company]. How can I help you today?", voice="alice")
    response.gather(input="speech", action="/process_speech", timeout=15)  # Wait for speech

    return Response(str(response), mimetype="application/xml")
@app.route("/process_speech", methods=["POST"])
def process_speech():
    print("Incoming Speech Data:", request.form)  # Log the request
    
    user_input = request.form.get("SpeechResult")  # Capture what Twilio heard
    print(f"User said: {user_input}")  # Print in console for debugging

    response = VoiceResponse()

    if user_input:
        response_text = retrieve_info(user_input)  # Process the input
    else:
        response_text = "I'm sorry, I didn't understand that."

    response.say(response_text, voice="alice")  # Twilio should speak back
    
    return Response(str(response), mimetype="application/xml")

if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Feb/2025 16:00:01] "POST /voice HTTP/1.1" 200 -


Incoming Speech Data: ImmutableMultiDict([('Called', '+16813666079'), ('ToState', 'WV'), ('CallerCountry', 'PK'), ('Direction', 'inbound'), ('SpeechResult', 'And me about your train over.  And you hear me?  Hello.  I need.'), ('CallerState', ''), ('Language', 'en-US'), ('ToZip', ''), ('Confidence', '0.52797556'), ('CallSid', 'CAf6b8f59408ae6db3ab8ace613ceae31a'), ('To', '+16813666079'), ('CallerZip', ''), ('ToCountry', 'US'), ('CalledZip', ''), ('ApiVersion', '2010-04-01'), ('CalledCity', ''), ('CallStatus', 'in-progress'), ('From', '+923464056076'), ('AccountSid', 'AC25ff99962f5f1c10f4610882da6d8652'), ('CalledCountry', 'US'), ('CallerCity', ''), ('ToCity', ''), ('FromCountry', 'PK'), ('Caller', '+923464056076'), ('FromCity', ''), ('CalledState', 'WV'), ('FromZip', ''), ('FromState', '')])
User said: And me about your train over.  And you hear me?  Hello.  I need.


127.0.0.1 - - [24/Feb/2025 16:00:46] "POST /process_speech HTTP/1.1" 200 -


In [ ]:
import os
import logging
import json
import uuid
import requests
from datetime import datetime
from flask import Flask, request, jsonify
from twilio.twiml.voice_response import VoiceResponse, Gather
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import speech_recognition as sr
import tempfile
from litellm import completion
import stripe
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import langsmith
from langsmith.run_helpers import traceable

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Flask app
app = Flask(__name__)

# Initialize environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
DOCUMENTS_DIR = os.environ.get("DOCUMENTS_DIR", "./documents")
STRIPE_API_KEY = os.environ.get("STRIPE_API_KEY")
EMAIL_USER = os.environ.get("EMAIL_USER")
EMAIL_PASSWORD = os.environ.get("EMAIL_PASSWORD")
EMAIL_SMTP_SERVER = os.environ.get("EMAIL_SMTP_SERVER", "smtp.gmail.com")
EMAIL_SMTP_PORT = int(os.environ.get("EMAIL_SMTP_PORT", 587))
CALENDLY_API_KEY = os.environ.get("CALENDLY_API_KEY")
CALENDLY_USER_URI = os.environ.get("CALENDLY_USER_URI")
LANGSMITH_API_KEY = os.environ.get("LANGSMITH_API_KEY")
LLM_MODEL = os.environ.get("LLM_MODEL", "gpt-4")
MINDWARE_API_KEY = os.environ.get("MINDWARE_API_KEY")
MINDWARE_ENDPOINT = os.environ.get("MINDWARE_ENDPOINT")

# Set up Stripe
if STRIPE_API_KEY:
    stripe.api_key = STRIPE_API_KEY

# Set up Langsmith
if LANGSMITH_API_KEY:
    langsmith.api_key = LANGSMITH_API_KEY

# Company and product information
COMPANY_INFO = {
    "name": "TechSolutions Inc.",
    "founded": 2015,
    "mission": "To help businesses grow through innovative technology solutions",
    "website": "www.techsolutions.com",
    "contact_email": "sales@techsolutions.com",
    "contact_phone": "+1-800-TECH-SOL"
}

PRODUCT_CATALOG = [
    {
        "id": "prod_001",
        "name": "Enterprise CRM",
        "description": "All-in-one customer relationship management platform",
        "price": 199.99,
        "billing": "monthly",
        "features": [
            "Contact management",
            "Sales pipeline tracking",
            "Email automation",
            "Analytics dashboard",
            "Mobile app"
        ],
        "use_cases": [
            "Sales teams looking to improve conversion rates",
            "Businesses wanting to streamline customer communication",
            "Companies seeking to centralize customer data"
        ]
    },
    {
        "id": "prod_002",
        "name": "Cloud Analytics Suite",
        "description": "Advanced data analytics and business intelligence platform",
        "price": 299.99,
        "billing": "monthly",
        "features": [
            "Real-time data processing",
            "Customizable dashboards",
            "Predictive analytics",
            "Data visualization tools",
            "Export capabilities"
        ],
        "use_cases": [
            "Data-driven decision making",
            "Market trend analysis",
            "Performance monitoring",
            "Customer behavior insights"
        ]
    }
]

# Sales stages definitions
SALES_STAGES = {
    "introduction": {
        "name": "Introduction",
        "description": "Introduce yourself and company",
        "example_phrases": [
            "Hi, I'm an AI sales representative from TechSolutions Inc.",
            "We help businesses like yours grow through innovative technology solutions."
        ],
        "next_stages": ["qualification"]
    },
    "qualification": {
        "name": "Qualification",
        "description": "Determine if prospect is right fit",
        "example_phrases": [
            "Are you currently responsible for managing customer relationships?",
            "What tools are you currently using for data analytics?"
        ],
        "next_stages": ["value_proposition", "needs_analysis"]
    },
    "value_proposition": {
        "name": "Value Proposition",
        "description": "Explain product/service benefits",
        "example_phrases": [
            "Our Enterprise CRM has helped similar companies increase sales by 27%.",
            "Our Cloud Analytics Suite can reduce reporting time by up to 85%."
        ],
        "next_stages": ["needs_analysis", "solution_presentation"]
    },
    "needs_analysis": {
        "name": "Needs Analysis",
        "description": "Uncover needs and pain points",
        "example_phrases": [
            "What challenges are you facing with your current system?",
            "How much time does your team spend on generating reports?"
        ],
        "next_stages": ["solution_presentation"]
    },
    "solution_presentation": {
        "name": "Solution Presentation",
        "description": "Present solution based on needs",
        "example_phrases": [
            "Based on what you've shared, our Enterprise CRM would address your challenges by...",
            "Our Cloud Analytics Suite would give you the insights you need because..."
        ],
        "next_stages": ["objection_handling", "close"]
    },
    "objection_handling": {
        "name": "Objection Handling",
        "description": "Address prospect's concerns",
        "example_phrases": [
            "I understand your concern about implementation time. We typically get clients up and running within just 2 weeks.",
            "While there is an upfront investment, our clients typically see ROI within 3 months."
        ],
        "next_stages": ["close", "solution_presentation"]
    },
    "close": {
        "name": "Close",
        "description": "Ask for the sale or next step",
        "example_phrases": [
            "Would you like to move forward with a subscription to our Enterprise CRM?",
            "Shall we schedule a demo with one of our implementation specialists?"
        ],
        "next_stages": ["end_conversation"]
    },
    "end_conversation": {
        "name": "End Conversation",
        "description": "Conclude the conversation",
        "example_phrases": [
            "Thank you for your time today. Feel free to reach out if you have any other questions.",
            "I appreciate you considering TechSolutions. Have a great day!"
        ],
        "next_stages": []
    }
}

# Session storage
call_sessions = {}

# Initialize RAG components
@traceable(run_type="chain", name="initialize_rag")
def initialize_rag():
    try:
        # Load company and product information
        with tempfile.NamedTemporaryFile(mode="w", delete=False, suffix=".txt") as temp_file:
            temp_file.write(json.dumps(COMPANY_INFO, indent=2))
            temp_file.write("\n\n")
            temp_file.write(json.dumps(PRODUCT_CATALOG, indent=2))
            company_info_path = temp_file.name
        
        # Load documents from directory and company info
        documents = []
        if os.path.exists(DOCUMENTS_DIR):
            loader = DirectoryLoader(DOCUMENTS_DIR)
            documents.extend(loader.load())
        
        # Add company info document
        company_loader = TextLoader(company_info_path)
        documents.extend(company_loader.load())
        
        # Clean up temp file
        os.unlink(company_info_path)
        
        logger.info(f"Loaded {len(documents)} documents")
        
        # Split documents into chunks
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        docs = text_splitter.split_documents(documents)
        logger.info(f"Split into {len(docs)} chunks")
        
        # Create embeddings and vector store
        embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
        vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)
        
        # Create retrieval chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=OpenAI(temperature=0.2, openai_api_key=OPENAI_API_KEY),
            chain_type="stuff",
            retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
        )
        
        return qa_chain
    except Exception as e:
        logger.error(f"Error initializing RAG: {e}")
        return None

# Initialize the RAG system
qa_system = initialize_rag()

# Function to get answer from LLM with context
@traceable(run_type="llm", name="get_sales_response")
def get_sales_response(session_id, query, current_stage):
    try:
        session = call_sessions.get(session_id, {
            "id": session_id,
            "conversation_history": [],
            "current_stage": "introduction",
            "customer_info": {},
            "identified_needs": [],
            "objections": [],
            "created_at": datetime.now().isoformat()
        })
        
        # Retrieve context from RAG
        context = ""
        if qa_system:
            context = qa_system.run(query)
        
        # Build prompt with context, conversation history, and sales stage
        system_prompt = f"""
        You are an AI sales agent for {COMPANY_INFO['name']}. 
        
        Current sales stage: {SALES_STAGES[current_stage]['name']}
        Stage description: {SALES_STAGES[current_stage]['description']}
        
        Company information:
        {json.dumps(COMPANY_INFO, indent=2)}
        
        Product catalog:
        {json.dumps(PRODUCT_CATALOG, indent=2)}
        
        Conversation history:
        {json.dumps(session['conversation_history'][-5:] if len(session['conversation_history']) > 0 else "No previous conversation", indent=2)}
        
        Customer info gathered so far:
        {json.dumps(session['customer_info'], indent=2)}
        
        Identified needs:
        {json.dumps(session['identified_needs'], indent=2)}
        
        Objections raised:
        {json.dumps(session['objections'], indent=2)}
        
        Retrieved context about query:
        {context}
        
        Be friendly, professional, and concise. Focus on understanding customer needs and how our products can solve their problems.
        Only reference products and features that are listed in the product catalog.
        Keep responses under 3 sentences for voice communication.
        """
        
        # Get response from LLM
        response = completion(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query}
            ],
            temperature=0.4,
            max_tokens=150
        )
        
        reply = response.choices[0].message.content
        
        # Update conversation history
        session['conversation_history'].append({
            "role": "customer",
            "content": query,
            "timestamp": datetime.now().isoformat()
        })
        session['conversation_history'].append({
            "role": "agent",
            "content": reply,
            "timestamp": datetime.now().isoformat()
        })
        
        call_sessions[session_id] = session
        
        return reply
    except Exception as e:
        logger.error(f"Error getting sales response: {e}")
        return "I apologize, but I'm having trouble processing that. Can we try a different approach?"

# Function to analyze conversation and update sales stage
@traceable(run_type="llm", name="analyze_conversation")
def analyze_conversation(session_id):
    try:
        session = call_sessions.get(session_id)
        if not session:
            return "introduction"
        
        # Get last few conversation turns
        recent_conversation = session['conversation_history'][-6:] if len(session['conversation_history']) >= 6 else session['conversation_history']
        current_stage = session.get('current_stage', 'introduction')
        
        # Build prompt for stage analysis
        system_prompt = f"""
        Analyze this conversation and determine the appropriate sales stage.
        
        Current stage: {current_stage}
        
        Possible next stages: {json.dumps(SALES_STAGES[current_stage]['next_stages'])}
        
        All sales stages:
        {json.dumps(SALES_STAGES, indent=2)}
        
        Recent conversation:
        {json.dumps(recent_conversation, indent=2)}
        
        Also identify:
        1. Any customer information (name, company, role, etc.)
        2. Customer needs or pain points
        3. Objections raised
        
        Output JSON only with the following structure:
        {{
            "next_stage": "stage_name",
            "customer_info": {{}},
            "identified_needs": [],
            "objections": []
        }}
        """
        
        # Get analysis from LLM
        response = completion(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": system_prompt}
            ],
            temperature=0.1,
            max_tokens=500,
            response_format={"type": "json_object"}
        )
        
        analysis = json.loads(response.choices[0].message.content)
        
        # Update session with analysis results
        session['current_stage'] = analysis['next_stage']
        session['customer_info'].update(analysis['customer_info'])
        session['identified_needs'].extend([need for need in analysis['identified_needs'] if need not in session['identified_needs']])
        session['objections'].extend([obj for obj in analysis['objections'] if obj not in session['objections']])
        
        call_sessions[session_id] = session
        
        return analysis['next_stage']
    except Exception as e:
        logger.error(f"Error analyzing conversation: {e}")
        return session.get('current_stage', 'introduction')

# Function to generate Stripe payment link
def generate_payment_link(product_id, customer_email=None):
    try:
        if not STRIPE_API_KEY:
            return None
        
        # Find product in catalog
        product = next((p for p in PRODUCT_CATALOG if p['id'] == product_id), None)
        if not product:
            return None
        
        # Create price in Stripe
        price = stripe.Price.create(
            unit_amount=int(product['price'] * 100),  # Convert to cents
            currency="usd",
            recurring={"interval": product['billing']} if product.get('billing') else None,
            product_data={
                "name": product['name'],
                "description": product['description']
            }
        )
        
        # Create payment link
        payment_link = stripe.PaymentLink.create(
            line_items=[{"price": price.id, "quantity": 1}],
            after_completion={"type": "redirect", "redirect": {"url": COMPANY_INFO['website']}}
        )
        
        return payment_link.url
    except Exception as e:
        logger.error(f"Error generating payment link: {e}")
        return None

# Function to send email
def send_email(to_email, subject, body):
    try:
        if not (EMAIL_USER and EMAIL_PASSWORD):
            return False
        
        # Create message
        msg = MIMEMultipart()
        msg['From'] = EMAIL_USER
        msg['To'] = to_email
        msg['Subject'] = subject
        
        # Add body
        msg.attach(MIMEText(body, 'html'))
        
        # Send email
        server = smtplib.SMTP(EMAIL_SMTP_SERVER, EMAIL_SMTP_PORT)
        server.starttls()
        server.login(EMAIL_USER, EMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        
        return True
    except Exception as e:
        logger.error(f"Error sending email: {e}")
        return False

# Function to generate Calendly link
def generate_calendly_link(event_type="30min"):
    try:
        if not (CALENDLY_API_KEY and CALENDLY_USER_URI):
            return "https://calendly.com/yourusername/30min"  # Fallback
        
        # Get user's event types
        headers = {
            "Authorization": f"Bearer {CALENDLY_API_KEY}",
            "Content-Type": "application/json"
        }
        
        response = requests.get(
            f"{CALENDLY_USER_URI}/event_types",
            headers=headers
        )
        
        if response.status_code == 200:
            event_types = response.json()['data']
            for event in event_types:
                if event_type in event['attributes']['slug']:
                    return event['attributes']['scheduling_url']
        
        return "https://calendly.com/yourusername/30min"  # Fallback
    except Exception as e:
        logger.error(f"Error generating Calendly link: {e}")
        return "https://calendly.com/yourusername/30min"  # Fallback

# Function to query Mindware
def query_mindware(query):
    try:
        if not (MINDWARE_API_KEY and MINDWARE_ENDPOINT):
            return None
        
        headers = {
            "Authorization": f"Bearer {MINDWARE_API_KEY}",
            "Content-Type": "application/json"
        }
        
        response = requests.post(
            MINDWARE_ENDPOINT,
            headers=headers,
            json={"query": query}
        )
        
        if response.status_code == 200:
            return response.json()
        
        return None
    except Exception as e:
        logger.error(f"Error querying Mindware: {e}")
        return None

# Twilio voice webhook endpoint
@app.route("/voice", methods=['POST'])
def voice():
    response = VoiceResponse()
    
    # Create new session
    session_id = str(uuid.uuid4())
    call_sessions[session_id] = {
        "id": session_id,
        "conversation_history": [],
        "current_stage": "introduction",
        "customer_info": {},
        "identified_needs": [],
        "objections": [],
        "created_at": datetime.now().isoformat()
    }
    
    # Initial greeting
    intro_message = f"Hello, this is an AI sales representative from {COMPANY_INFO['name']}. How can I help you today?"
    response.say(intro_message)
    
    # Update conversation history
    call_sessions[session_id]['conversation_history'].append({
        "role": "agent",
        "content": intro_message,
        "timestamp": datetime.now().isoformat()
    })
    
    # Gather speech input
    gather = Gather(input='speech', action=f'/process_speech?session_id={session_id}', timeout=5, speechTimeout='auto')
    response.append(gather)
    
    # If user doesn't say anything
    response.say("I didn't hear you. Please let me know how I can help you today.")
    
    return str(response)

# Process speech endpoint
@app.route("/process_speech", methods=['POST'])
def process_speech():
    response = VoiceResponse()
    
    # Get session ID
    session_id = request.args.get('session_id')
    if not session_id or session_id not in call_sessions:
        # Create new session if missing
        session_id = str(uuid.uuid4())
        call_sessions[session_id] = {
            "id": session_id,
            "conversation_history": [],
            "current_stage": "introduction",
            "customer_info": {},
            "identified_needs": [],
            "objections": [],
            "created_at": datetime.now().isoformat()
        }
    
    # Get the speech input
    speech_result = request.values.get('SpeechResult')
    
    if speech_result:
        logger.info(f"Received speech: {speech_result}")
        
        # Analyze conversation to determine stage
        current_stage = analyze_conversation(session_id)
        
        # Get response based on current stage
        agent_response = get_sales_response(session_id, speech_result, current_stage)
        
        # Special handling for close stage
        if current_stage == "close":
            session = call_sessions[session_id]
            if any(need in speech_result.lower() for need in ["buy", "purchase", "payment", "sign up"]):
                # Customer wants to purchase
                product_id = "prod_001"  # Default product
                for product in PRODUCT_CATALOG:
                    if product['name'].lower() in " ".join(session['conversation_history']).lower():
                        product_id = product['id']
                        break
                
                payment_link = generate_payment_link(product_id)
                if payment_link:
                    agent_response += f" I've generated a payment link for you. I'll send it to your email shortly. Is there anything else you'd like to know?"
                    
                    # Send email if we have customer email
                    customer_email = session['customer_info'].get('email')
                    if customer_email:
                        email_body = f"""
                        <html>
                        <body>
                            <h2>Thank you for your interest in {COMPANY_INFO['name']}!</h2>
                            <p>As discussed, here is your payment link for the {next((p['name'] for p in PRODUCT_CATALOG if p['id'] == product_id), 'product')}:</p>
                            <p><a href="{payment_link}">Complete your purchase</a></p>
                            <p>If you have any questions, please contact us at {COMPANY_INFO['contact_email']}.</p>
                            <p>Best regards,<br>The {COMPANY_INFO['name']} Team</p>
                        </body>
                        </html>
                        """
                        send_email(customer_email, f"Your {COMPANY_INFO['name']} Payment Link", email_body)
            
            elif any(word in speech_result.lower() for word in ["call", "meeting", "schedule", "demo"]):
                # Customer wants to schedule a meeting
                calendly_link = generate_calendly_link()
                agent_response += f" I can help you schedule a demo. I'll send a calendar link to your email. Is there anything else I can assist you with?"
                
                # Send email if we have customer email
                customer_email = session['customer_info'].get('email')
                if customer_email:
                    email_body = f"""
                    <html>
                    <body>
                        <h2>Schedule Your Demo with {COMPANY_INFO['name']}</h2>
                        <p>Thank you for your interest in our products!</p>
                        <p>Please use the link below to schedule a time that works best for you:</p>
                        <p><a href="{calendly_link}">Schedule Your Demo</a></p>
                        <p>If you have any questions, please contact us at {COMPANY_INFO['contact_email']}.</p>
                        <p>Best regards,<br>The {COMPANY_INFO['name']} Team</p>
                    </body>
                    </html>
                    """
                    send_email(customer_email, f"Schedule Your {COMPANY_INFO['name']} Demo", email_body)
        
        # Respond to the user
        response.say(agent_response)
    else:
        response.say("I couldn't understand what you said. Could you please repeat that?")
    
    # Check if we should end the conversation
    session = call_sessions[session_id]
    if session.get('current_stage') == "end_conversation":
        response.say("Thank you for your time today. Have a great day!")
        response.hangup()
    else:
        # Gather next input
        gather = Gather(input='speech', action=f'/process_speech?session_id={session_id}', timeout=5, speechTimeout='auto')
        response.append(gather)
        
        # If user doesn't respond
        response.say("I didn't hear you. Please let me know if there's anything else I can help with.")
        
    return str(response)

# API endpoint to get session data
@app.route("/api/sessions/<session_id>", methods=['GET'])
def get_session(session_id):
    if session_id in call_sessions:
        return jsonify(call_sessions[session_id])
    return jsonify({"error": "Session not found"}), 404

# API endpoint for human handoff
@app.route("/api/handoff/<session_id>", methods=['POST'])
def human_handoff(session_id):
    if session_id not in call_sessions:
        return jsonify({"error": "Session not found"}), 404
    
    # Update session to indicate human handoff
    call_sessions[session_id]['human_handoff'] = True
    call_sessions[session_id]['handoff_timestamp'] = datetime.now().isoformat()
    
    # Additional logic for human handoff would go here (e.g., notifying a human agent)
    
    return jsonify({"success": True, "message": "Handoff initiated"})

if __name__ == "__main__":
    # Check if RAG system is initialized
    if not qa_system:
        logger.warning("RAG system failed to initialize. The application will still run but won't provide accurate answers.")
    
    # Run the Flask app
    app.run(debug=True, host='0.0.0.0', port=5000)